In [1]:
from nipype.interfaces.io import DataSink, SelectFiles, DataGrabber 
from nipype.interfaces.utility import IdentityInterface, Function    
from nipype.pipeline.engine import Node, Workflow, JoinNode, MapNode
from nipype.interfaces import fsl
from nipype.interfaces import freesurfer as fsr
from nipype.interfaces import brainsuite
from nipype.interfaces.io import SelectFiles, DataSink, DataGrabber
from shutil import copyfile
import pandas as pd
from glob import glob
from os.path import abspath, expanduser, join
from os import chdir, remove, getcwd
from nipype import config, logging
from datetime import date
today = str(date.today())
config.enable_debug_mode()

In [2]:
#Set variables
# subject_list = ['02-T2', '06-T1', '07-T2','08-T1', '09-T2', '10-T1', '11-T1', '12-T1', '13-T1', '15-T1', '16-T1', '17-T1',
#                 '18-T1', '19-T1', '20-T1', '21-T1', '22-T1', '23-T1', '24-T1', '29-T1', '30-T1', '31-T1', '32-T1',
#                 '35-T1', '36-T1', '37-T1', '38-T1', '40-T1', '102-T1', '103-T1', '104-T1', '105-T1', '106-T1', '107-T1', '108-T1', '109-T1',
#                 '110-T1', '111-T1']

#subject_list = ['08-T1']#, '20-T1','22-T1','29-T1','31-T1','35-T1']

home = '/Volumes/group/proc/TIGERanalysis'
fpath = join(home, 'qT1/subjDir')
workflow_dir = join(home, 'qT1/workflow')
data_dir = join(home, 'qT1/data')
data = '/Users/lucindasisk/Box/LS_Folders/TIGER_qT1_SOBP'

# subs = glob(data + '/*-*')
# subject_list = pd.Series(subs).str.replace(data + '/', '')
# subject_list

subs = pd.read_csv(home + '/qT1/scripts/qt1_unfinishedSubcort_04.02.20.txt', sep = ' ', header=None)
subject_list = subs[0].tolist()

#subject_list = ['097-T3']

In [3]:
#Setup Datasink, Infosource, Selectfiles

datasink = Node(DataSink(base_directory = data_dir),
                   name='datasink')

#Set infosource iterables
infosource = Node(IdentityInterface(fields=['subject_id']),
                  name="infosource")
infosource.iterables = [('subject_id', subject_list)]

#SelectFiles
template = dict(t1=fpath+'/{subject_id}/T1_acpc.nii.gz',
               qt1=data+'/{subject_id}/fw_qT1_t1fit_final.nii.gz')

sf = Node(SelectFiles(template, 
                      base_directory = fpath),
          name = 'sf')


In [ ]:
#Skullstrip T1w image
stripT1 = Node(fsl.BET(out_file = 'T1_acpc_bet.nii.gz'),
             name = 'stripT1')

#Reorient qT1 to standard orientation
reorient = Node(fsl.Reorient2Std(out_file = 'reoriented_qT1.nii.gz',
                              output_type = 'NIFTI_GZ'),
                    name = 'reorient')

#Register qT1 to T1w
register_qt1 = Node(fsl.FLIRT(out_file = 'registered_qT1_T1acpc.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt1')

#Extract voxel values from qT1 beneath overlaid mask - Left Uncinate
get_voxLUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftUncinate.txt'
                               out_file = 'ROI_voxel-contrasts_LeftCC.txt'),
               name = 'get_voxLCC')),
               name = 'get_voxLUF')

#Extract voxel values from qT1 beneath overlaid mask - Left Cingulum Cingulate
get_voxLCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCC.txt'),
               name = 'get_voxLCC')

#Extract voxel values from qT1 beneath overlaid mask - Right Uncinate
get_voxRUF = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightHipp.txt'),
               name = 'get_voxRUF')

#Extract voxel values from qT1 beneath overlaid mask - Right Cingulum Cingulate
get_voxRCC = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftHipp.txt'),
               name = 'get_voxRCC')

In [ ]:
qt1proc_flow = Workflow(name='qt1proc_flow')
qt1proc_flow.connect([(infosource, sf, [('subject_id', 'subject_id')]),
                      (sf, stripT1, [('t1', 'in_file')]),
                      (stripT1, datasink, [('out_file', '1_check_alignment')]),
                      (sf, reorient, [('qt1', 'in_file')]),
                      (reorient, datasink, [
                       ('out_file', '1_check_alignment.@par')]),
                      (reorient, register_qt1, [('out_file', 'in_file')]),
                      (stripT1, register_qt1, [('out_file', 'reference')]),
                      (register_qt1, datasink, [
                       ('out_file', '1_check_alignment.@par.@par')])
                      ])

qt1proc_flow.base_dir = workflow_dir
qt1proc_flow.write_graph(graph2use='flat')
preproc = qt1proc_flow.run('MultiProc', plugin_args={'n_procs': 4})

In [ ]:
#SelectFiles
template2 = dict(qt1r=data_dir+'/1_check_alignment/_subject_id_{subject_id}/registered_qT1.nii.gz',
               unc_maskL = fpath+'/{subject_id}/Left_Uncinate.nii.gz',
               unc_maskR = fpath+'/{subject_id}/Right_Uncinate.nii.gz',
               cc_maskL = fpath+'/{subject_id}/Left_Cingulum_Cingulate.nii.gz',
               cc_maskR = fpath+'/{subject_id}/Right_Cingulum_Cingulate.nii.gz')

# 
#             bval=fpath+'/{subject_id}/dti.bval',
#             bvec=fpath+'/{subject_id}/dti.bvec'

sf2 = Node(SelectFiles(template2, 
                      base_directory = fpath),
          name = 'sf2')

In [ ]:
get_stats=Workflow(name='get_stats')
get_stats.connect([(infosource, sf2, [('subject_id','subject_id')]),
                   (sf2, get_voxLUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxLUF, [('unc_maskL', 'mask')]),
                   (get_voxLUF, datasink, [('out_file', '2_Extracted_Stats')]),
                   (sf2, get_voxLCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxLCC, [('cc_maskL', 'mask')]),
                   (get_voxLCC, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                   (sf2, get_voxRUF, [('qt1r', 'in_file')]),
                   (sf2, get_voxRUF, [('unc_maskR', 'mask')]),
                   (get_voxRUF, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                   (sf2, get_voxRCC, [('qt1r', 'in_file')]),
                   (sf2, get_voxRCC, [('cc_maskR', 'mask')]),
                   (get_voxRCC, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')])
                  ])
get_stats.base_dir = workflow_dir
get_stats.write_graph(graph2use = 'flat')
preproc = get_stats.run('MultiProc', plugin_args={'n_procs': 4})

In [4]:
#SelectFiles
template3 = dict(t1w = fpath+'/{subject_id}/freesurfer_brain.nii.gz',
                aseg = fpath+'/{subject_id}/aparc.a2009s+aseg.nii.gz',
                qt1_reor = data_dir+'/1_check_alignment/_subject_id_{subject_id}/reoriented_qT1.nii.gz')       


sf3 = Node(SelectFiles(template3, 
                      base_directory = fpath),
          name = 'sf3')

In [5]:
# #MRI Convert Nodes
# convert_wb = Node(fsr.MRIConvert(in_type='mgz',
#                                out_file = 'freesurfer_brain.nii.gz'),
#                name = 'convert_wb')

# convert_aseg = Node(fsr.MRIConvert(in_type='mgz',
#                                out_file = 'aparc.a2009s+aseg.nii.gz'),
#                name = 'convert_aseg')


In [6]:
#Extract left hippocampus - seg value 17
extract_lhipp = Node(fsl.Threshold(thresh = 16.5, 
                                   direction = 'below', 
                                   args = '-uthr 17.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_hippocampus.nii.gz'),
                        name = 'extract_lhipp')
#Extract right hippocampus - seg value 53
extract_rhipp = Node(fsl.Threshold(thresh = 52.5, 
                                   direction = 'below', 
                                   args = '-uthr 53.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_hippocampus.nii.gz'),
                    name='extract_rhipp')


In [7]:
#Extract left amygdala - seg value 18
extract_lamyg = Node(fsl.Threshold(thresh = 17.5, 
                                   direction = 'below', 
                                   args = '-uthr 18.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_amygdala.nii.gz'),
                        name = 'extract_lamyg')
#Extract right amygdala - seg value 54
extract_ramyg = Node(fsl.Threshold(thresh = 53.5, 
                                   direction = 'below', 
                                   args = '-uthr 54.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_amygdala.nii.gz'),
                    name='extract_ramyg')

In [8]:
#Extract left caudate - seg value 11
extract_lcaud = Node(fsl.Threshold(thresh = 10.5, 
                                   direction = 'below', 
                                   args = '-uthr 11.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_caudate.nii.gz'),
                        name = 'extract_lcaud')

#Extract right caudate - seg value 50
extract_rcaud = Node(fsl.Threshold(thresh = 49.5, 
                                   direction = 'below', 
                                   args = '-uthr 50.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_caudate.nii.gz'),
                    name='extract_rcaud')


In [9]:
#Extract left nucleus accumbens - seg value 26
extract_lnacc  = Node(fsl.Threshold(thresh = 25.5, 
                                   direction = 'below', 
                                   args = '-uthr 26.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_nAcc.nii.gz'),
                        name = 'extract_lnacc')

#Extract right caudate - seg value 58
extract_rnacc = Node(fsl.Threshold(thresh = 57.5, 
                                   direction = 'below', 
                                   args = '-uthr 58.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_nAcc.nii.gz'),
                    name='extract_rnacc')


In [10]:
#Extract left putamen - seg value 26
extract_lput  = Node(fsl.Threshold(thresh = 11.5, 
                                   direction = 'below', 
                                   args = '-uthr 12.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'left_putamen.nii.gz'),
                        name = 'extract_lput')

#Extract right caudate - seg value 51
extract_rput = Node(fsl.Threshold(thresh = 50.5, 
                                   direction = 'below', 
                                   args = '-uthr 51.5',
                                   output_type = 'NIFTI_GZ',
                                   out_file = 'right_putamen.nii.gz'),
                    name='extract_rput')

In [11]:
#Extract voxel values from qT1 beneath overlaid mask - Left Hippocampus
get_voxRhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightHippocampus.txt'),
               name = 'get_voxRhipp')

#Extract voxel values from qT1 beneath overlaid mask - Right Hippocampus
get_voxLhipp = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftHippocampus.txt'),
               name = 'get_voxLhipp')

In [12]:
#Extract voxel values from qT1 beneath overlaid mask - Left Amygdala
get_voxRamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightAmygdala.txt'),
               name = 'get_voxRamyg')

#Extract voxel values from qT1 beneath overlaid mask - Right Amygdala
get_voxLamyg = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftAmygdala.txt'),
               name = 'get_voxLamyg')

In [13]:
#Extract voxel values from qT1 beneath overlaid mask - Left Putamen
get_voxRput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightPutamen.txt'),
               name = 'get_voxRput')

#Extract voxel values from qT1 beneath overlaid mask - Right Putamen
get_voxLput = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftPutamen.txt'),
               name = 'get_voxLput')

In [14]:
#Extract voxel values from qT1 beneath overlaid mask - Left NAcc
get_voxRnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightNAcc.txt'),
               name = 'get_voxRnacc')

#Extract voxel values from qT1 beneath overlaid mask - Right NAcc
get_voxLnacc = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftNAcc.txt'),
               name = 'get_voxLnacc')

In [15]:
#Extract voxel values from qT1 beneath overlaid mask - Left Caudate
get_voxRcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_RightCaudate.txt'),
               name = 'get_voxRcaud')

#Extract voxel values from qT1 beneath overlaid mask - Right Caudate
get_voxLcaud = Node(fsl.ImageMeants(show_all = True,
                               out_file = 'ROI_voxel-contrasts_LeftCaudate.txt'),
               name = 'get_voxLcaud')

In [16]:
#Register qT1 to T1w
register_qt12 = Node(fsl.FLIRT(out_file = 'registered_qT1_toFreesurfer.nii.gz',
                              output_type = 'NIFTI_GZ',
                              out_matrix_file = 'flirt_transform_qt1.mat'),
                    name = 'register_qt12')

In [19]:
hippdata_flow = Workflow(name='hippdata_flow')
hippdata_flow.connect([(infosource, sf3, [('subject_id', 'subject_id')]),
                       (sf3, register_qt12, [('qt1_reor', 'in_file')]),
                       (sf3, register_qt12, [('t1w', 'reference')]),
                       (sf3, extract_lhipp, [('aseg', 'in_file')]),
                       (sf3, extract_rhipp, [('aseg', 'in_file')]),
                       (sf3, extract_lamyg, [('aseg', 'in_file')]),
                       (sf3, extract_ramyg, [('aseg', 'in_file')]),
                       (sf3, extract_lput, [('aseg', 'in_file')]),
                       (sf3, extract_rput, [('aseg', 'in_file')]),
                       (sf3, extract_lnacc, [('aseg', 'in_file')]),
                       (sf3, extract_rnacc, [('aseg', 'in_file')]),
                       (sf3, extract_lcaud, [('aseg', 'in_file')]),
                       (sf3, extract_rcaud, [('aseg', 'in_file')]),
                       (extract_lhipp, datasink, [('out_file', '1_check_alignment')]),
                       (extract_rhipp, datasink, [('out_file', '1_check_alignment.@par')]),
                       (extract_lamyg, datasink, [('out_file', '1_check_alignment.@par.@par')]),
                       (extract_ramyg, datasink, [('out_file', '1_check_alignment.@par.@par.@par')]),
                       (extract_lput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par')]),
                       (extract_rput, datasink, [('out_file', '1_check_alignment.@par.@par.@par.@par.@par')]),
                       (extract_lnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par')]),
                       (extract_rnacc, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_lcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (extract_rcaud, datasink, [('out_file', 
                                                   '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, datasink,[('out_file', '1_check_alignment.@par.@par.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (register_qt12, get_voxRhipp, [('out_file', 'in_file')]),
                       (extract_rhipp, get_voxRhipp, [('out_file', 'mask')]),
                       (register_qt12, get_voxLhipp, [('out_file', 'in_file')]),
                       (extract_lhipp, get_voxLhipp, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRamyg, [('out_file', 'in_file')]),
                       (extract_ramyg, get_voxRamyg, [('out_file', 'mask')]),
                       (register_qt12, get_voxLamyg, [('out_file', 'in_file')]),
                       (extract_lamyg, get_voxLamyg, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRput, [('out_file', 'in_file')]),
                       (extract_rput, get_voxRput, [('out_file', 'mask')]),
                       (register_qt12, get_voxLput, [('out_file', 'in_file')]),
                       (extract_lput, get_voxLput, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRcaud, [('out_file', 'in_file')]),
                       (extract_rcaud, get_voxRcaud, [('out_file', 'mask')]),
                       (register_qt12, get_voxLcaud, [('out_file', 'in_file')]),
                       (extract_lcaud, get_voxLcaud, [('out_file', 'mask')]),
                       
                       (register_qt12, get_voxRnacc, [('out_file', 'in_file')]),
                       (extract_rnacc, get_voxRnacc, [('out_file', 'mask')]),
                       (register_qt12, get_voxLnacc, [('out_file', 'in_file')]),
                       (extract_lnacc, get_voxLnacc, [('out_file', 'mask')]),
                       
                       (get_voxLhipp, datasink, [('out_file', '2_Extracted_Stats')]),
                       (get_voxRhipp, datasink, [('out_file', '2_Extracted_Stats.@par')]),
                       (get_voxLamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par')]),
                       (get_voxRamyg, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par')]),
                       (get_voxLput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par')]),
                       (get_voxRput, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par')]),
                       (get_voxLnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRnacc, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxLcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par')]),
                       (get_voxRcaud, datasink, [('out_file', '2_Extracted_Stats.@par.@par.@par.@par.@par.@par.@par.@par.@par')])
                      ])

hippdata_flow.base_dir = workflow_dir
hippdata_flow.write_graph(graph2use='flat')
preproc = hippdata_flow.run('MultiProc', plugin_args={'n_procs': 4})

200407-21:55:06,377 nipype.workflow DEBUG:
	 (hippdata_flow.infosource, hippdata_flow.sf3): No edge data
200407-21:55:06,378 nipype.workflow DEBUG:
	 (hippdata_flow.infosource, hippdata_flow.sf3): new edge data: {'connect': [('subject_id', 'subject_id')]}
200407-21:55:06,379 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): No edge data
200407-21:55:06,380 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): new edge data: {'connect': [('qt1_reor', 'in_file')]}
200407-21:55:06,381 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): Edge data exists: {'connect': [('qt1_reor', 'in_file')]}
200407-21:55:06,382 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.register_qt12): new edge data: {'connect': [('qt1_reor', 'in_file'), ('t1w', 'reference')]}
200407-21:55:06,383 nipype.workflow DEBUG:
	 (hippdata_flow.sf3, hippdata_flow.extract_lhipp): No edge data
200407-21:55:06,384 nipype.workflow DEBUG:
	 (hippdata_f

200407-21:55:06,419 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxLamyg): No edge data
200407-21:55:06,420 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxLamyg): new edge data: {'connect': [('out_file', 'in_file')]}
200407-21:55:06,420 nipype.workflow DEBUG:
	 (hippdata_flow.extract_lamyg, hippdata_flow.get_voxLamyg): No edge data
200407-21:55:06,421 nipype.workflow DEBUG:
	 (hippdata_flow.extract_lamyg, hippdata_flow.get_voxLamyg): new edge data: {'connect': [('out_file', 'mask')]}
200407-21:55:06,421 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxRput): No edge data
200407-21:55:06,422 nipype.workflow DEBUG:
	 (hippdata_flow.register_qt12, hippdata_flow.get_voxRput): new edge data: {'connect': [('out_file', 'in_file')]}
200407-21:55:06,423 nipype.workflow DEBUG:
	 (hippdata_flow.extract_rput, hippdata_flow.get_voxRput): No edge data
200407-21:55:06,423 nipype.workflow DEBUG:
	 (hippdata_flow.extra

200407-21:55:06,899 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRnacc
200407-21:55:06,900 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLcaud
200407-21:55:06,901 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRcaud
200407-21:55:06,902 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLput
200407-21:55:06,902 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRput
200407-21:55:06,903 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLamyg
200407-21:55:06,904 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRamyg
200407-21:55:06,904 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxLhipp
200407-21:55:06,905 nipype.workflow DEBUG:
	 processing node: hippdata_flow.get_voxRhipp
200407-21:55:06,905 nipype.workflow DEBUG:
	 processing node: hippdata_flow.datasink
200407-21:55:06,906 nipype.workflow DEBUG:
	 finished expanding workflow: hippdata_flow
200407-21:55:06,913 nipype.w

200407-21:55:39,24 nipype.workflow DEBUG:
	 Tasks currently running: 1. Pending: 1.
200407-21:55:39,26 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 0 jobs ready. Free memory (GB): 28.60/28.80, Free processors: 3/4.
                     Currently running:
                       * hippdata_flow.sf3
200407-21:55:45,174 nipype.utils DEBUG:
	 Creating directory /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/sf3/_report
200407-21:55:45,468 nipype.workflow DEBUG:
	 [Node] Writing pre-exec report to "/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/sf3/_report/report.rst"
200407-21:55:50,658 nipype.workflow INFO:
	 [Node] Running "sf3" ("nipype.interfaces.io.SelectFiles")
200407-21:55:53,953 nipype.workflow DEBUG:
	 Needed files: /Volumes/group/proc/TIGERanalysis/qT1/subjDir/021-T3/freesurfer_brain.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/subjDir/021-T3/aparc.a2009s+aseg.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1

	 [Node] Hashes: [('args', '-uthr 11.5'), ('direction', 'below'), ('in_file', ('/Volumes/group/proc/TIGERanalysis/qT1/subjDir/021-T3/aparc.a2009s+aseg.nii.gz', '170ef3cc2b0a1cb152e4785d64d001af')), ('out_file', 'left_caudate.nii.gz'), ('output_type', 'NIFTI_GZ'), ('thresh', '10.5000000000')], b36be1cbe03753bb70f06675bd0198cb, /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_lcaud/_0xb36be1cbe03753bb70f06675bd0198cb.json, []
200407-21:56:01,132 nipype.workflow DEBUG:
	 [Node] No hashfiles found in "/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_lcaud".
200407-21:56:01,139 nipype.utils DEBUG:
	 Removing contents of /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_lcaud
200407-21:56:01,910 nipype.workflow DEBUG:
	 [Node] Hashes: [('args', '-uthr 58.5'), ('direction', 'below'), ('in_file', ('/Volumes/group/proc/TIGERanalysis/qT1/subjDir/021-T3/aparc.a2009s+aseg.nii.gz', '1

...

	 key: 1_check_alignment.@par.@par.@par.@par.@par files: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_rput/right_putamen.nii.gz
200407-22:11:20,834 nipype.interface DEBUG:
	 copyfile: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_rput/right_putamen.nii.gz /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_putamen.nii.gz
200407-22:11:20,840 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_putamen.nii.gz
200407-22:11:20,934 nipype.utils DEBUG:
	 Linking File: /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_putamen.nii.gz->/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/extract_rput/right_putamen.nii.gz
200407-22:11:21,127 nipype.utils DEBUG:
	 Copying File: /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_p

	 File: /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_caudate.nii.gz already exists,737060841fc705e790d2406f2bf44d73, copy:1
200407-22:11:42,281 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_caudate.nii.gz already exists, not overwriting, copy:1
200407-22:11:42,287 nipype.interface DEBUG:
	 key: 2_Extracted_Stats.@par files: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRhipp/ROI_voxel-contrasts_RightHippocampus.txt
200407-22:11:42,594 nipype.interface DEBUG:
	 copyfile: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRhipp/ROI_voxel-contrasts_RightHippocampus.txt /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightHippocampus.txt
200407-22:11:42,600 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T

	 copyfile: /Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRput/ROI_voxel-contrasts_RightPutamen.txt /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightPutamen.txt
200407-22:12:04,787 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightPutamen.txt
200407-22:12:04,883 nipype.utils DEBUG:
	 Linking File: /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightPutamen.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRput/ROI_voxel-contrasts_RightPutamen.txt
200407-22:12:05,82 nipype.utils DEBUG:
	 Copying File: /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightPutamen.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRput/ROI_voxel-contr

	 Linking File: /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightNAcc.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRnacc/ROI_voxel-contrasts_RightNAcc.txt
200407-22:12:33,945 nipype.utils DEBUG:
	 Copying File: /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightNAcc.txt->/Volumes/group/proc/TIGERanalysis/qT1/workflow/hippdata_flow/_subject_id_021-T3/get_voxRnacc/ROI_voxel-contrasts_RightNAcc.txt
200407-22:12:35,732 nipype.utils DEBUG:
	 /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightNAcc.txt
200407-22:12:36,113 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TIGERanalysis/qT1/data/2_Extracted_Stats/_subject_id_021-T3/ROI_voxel-contrasts_RightNAcc.txt already exists,cb4b78fd03e8a6ae8bc464b949b66a08, copy:1
200407-22:12:36,495 nipype.utils DEBUG:
	 File: /Volumes/group/proc/TI

	 Needed files: /Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/registered_qT1_toFreesurfer.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/left_hippocampus.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_hippocampus.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/left_amygdala.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_amygdala.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/left_putamen.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_putamen.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/left_nAcc.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/_subject_id_021-T3/right_nAcc.nii.gz;/Volumes/group/proc/TIGERanalysis/qT1/data/1_check_alignment/

In [17]:
subject_list

['021-T3']